## Gaussian process analysis of X-ray time lags

This notebook uses the pyLag X-ray spectral timing package to perform spectral-timing analysis on unevenly sampled light curves, or light curves with gaps. We will estimate the lag-frequency and lag-energy spectra from unevenly sampled light curves by fitting a Gaussian process to the light curve in each energy band. Once the Gaussian processes have been fitted to the observed light curve data points, samples of the underlying continuous time series are drawn, and a single sample of the lag-energy spectrum is calculated from these sampled light curves. The mean lag-energy spectrum and its errors are estimated from the calculated sample. 

This notebook follows the Gaussian process method presented in Wilkins 2019, MNRAS 489, 1957 (https://arxiv.org/abs/1908.06099). Please cite this paper if you make use of any of these methods.

Requires the pylag package to be installed, in addition to scikit-learn for the Gaussian process engine (in addition to their dependencies).

D.R. Wilkins (August 2024)

### Fitting the Gaussian process to a pair of light curves light curves for the lag-frequency spectrum

The first step is to fit the Gaussian process model to each light curve we wish to use for the spectral timing analysis. We can fit a Gaussian process to a light curve using the GPLightCurve class. This class provides all of the same functionality as a standard LightCurve, but with the capability to perform the Gaussian process modelling.

We need to select the Gaussian process kernel to use, which describes the autocorrelation function of the intrinsic variability. Depending on the length of the gaps in the list curves and the timescales of interest, the most appropriate options are either 'rq' (the rational quadratic kernel) or 'matern12' (the Matern-1/2 kernel), although any kernel function from scikit-learn can be used.

We can choose how to model the uncorrelated noise between each time bin (i.e. the Poisson noise). We can either directly use the error bars (use_errors=True) or we can add the noise term into the kernel function (noise_kernel=True). Note that these options are mutually exclusive.

We can also select whether to model the light curve as the log of the count rate in each time bin (lognorm=True), which may be appropriate for systems in which the variability follows a log-normal distribution.

The remove_nan and remove_gaps options clean up any zero-count time bins in the original light curves that may be in place of the gaps (the light curves that are passed to the Gaussian process classes should simply have the time bins missing where no data are available).

The run_fit=True option will run the Gaussian process model fit for each light curve immediately.

The GPLightCurve class can either read the light curve directly from a FITS file, or it can be passed an existing LightCurve object with the lc=my_lightcurve argument. This allows us to perform various operations on the light curve before fitting, such as concatenating segmetns from multiple observations into a single long light curve.

In [ ]:
gplc1 = GPLightCurve('lightcurves/lc1.lc', kernel='rq', lognorm=False, noise_kernel=True, use_errors=False, remove_gaps=True, remove_nan=True, run_fit=True)
gplc2 = GPLightCurve('lightcurves/lc2.lc', kernel='rq', lognorm=False, noise_kernel=True, use_errors=False, remove_gaps=True, remove_nan=True, run_fit=True)

We can plot the original light curve simply using Plot(gplc1), or we can draw a single sample from the Gaussian process model using gplc1.sample(n_samples=1), which we can then plot like any other LightCurve object. We can also marginalise the prediction for the light curve in the gaps over many samples using gplc1.predict(). Plotting these on top of each other to see how the Gaussian process predicts the underlying variability:

In [ ]:
sample_lc = gplc1.sample(n_samples=1)
prediction_lc = gplc1.predict()
Plot([gplc1, sample_lc, prediction_lc], fig_size=(10,3))

## Sampling the lag-frequency spectrum

Once the Gaussian process models have been fit to the pair of light curves, we can use them to sample the lag-frequency spectrum. The GPLagFrequencySpectrum class will automatically draw samples from each of the Gaussian process light curve models, then compute the time lags between them, repeating the process to obtain samples of the lag spectrum.

GPLagFrequencySpectrum takes many of the same options as the standard LagFrequencySpectrum class. We specify the frequency bins in which we wish to calculate the lags, and the number of samples we want to use to estimate the mean and error of the lags.

The low_mem option will calculate the lag spectrum of the samples one at a time. Turning this off will calculate the samples in parallel, which requires much more memory (although is faster).

In [ ]:
gplf = GPLagFrequencySpectrum(fbins, gplc1=gplc1, gplc2=gplc2, n_samples=1000, low_mem=True)

Now we can plot the results and save them in a text file

In [ ]:
Plot(gplf)
write_data(gple, 'gplf_results.dat')

### Fitting the Gaussian process to light curves for the lag-energy spectrum

We will need to load a list of light curves in each energy band. This is handled by the EnergyLCList object, which reads a collection of FITS light curve files for different energy bands, collected over a number of observations (each individual file should be the light curve in a specific energy band from one observation). The energy corresponding to each light curve is read from the filename (see the pyLag documentation for details of how the files should be named). We also specify the frequency range over which to calculate the lag-energy spectrum. We will change this interactively, but we just need to specify some starting values to get everything set up.

Once we have loaded the light curves, we can fit a Gaussian process to each one using the GPEnergyLCList object. We need to select the Gaussian process kernel to use, and can set the other options for handling noise and pre-processing the light curve as above.

The run_fit=True option will run the Gaussian process model fit for each light curve immediately.

In [7]:
lclist = EnergyLCList('lightcurves/for_lagen/*.lc', interp_gaps=True)

In [ ]:
gplclclist = GPEnergyLCList(enlclist=lclist, kernel='rq', lognorm=False, noise_kernel=True, use_errors=False, remove_gaps=True, remove_nan=True, run_fit=True)

### Sampling the lag-energy spectrum

Now that we have fit the Gaussian process model to the light curves in each energy band, we can use them to sample the lag-energy spectrum. This GPLagEnergySpectrum class will automatically draw samples from each of the Gaussian process light curve models, then compute the time lags between them, repeating the process to obtain samples of the lag spectrum.

GPLagEnergySpectrum takes many of the same options as the standard LagEnergySpectrum class. We specify the frequency interval over which we wish to calculate the time lags (fmin and fmax), and the number of samples we want to use to estimate the mean and error of the lags. As in a conventional Fourier spectral timing analysis, we may with to use the lag-frequency spectrum to estimate the frequency range in which the reverberation appears.

The low_mem option will calculate the lag spectrum of the samples one at a time. Turning this off will calculate the samples in parallel, which requires *much* more memory (although is faster).

The save_samples option will store the individual lag-energy spectra from each set of samples drawn from the Gaussian processes in an array. This can be useful for parallelising the calculation of many sets of samples, or to perform further statistical analysis on the population of samples. If this option is off, only the mean and error will be stored in the gple.lag and gple.error array.

In [ ]:
fmin, fmax = 1e-4, 1e-3
gple = GPLagEnergySpectrum(fmin, fmax, gplclist=gplclist, n_samples=1000, low_mem=True, save_samples=False)

Againn we can plot the resuls and save them in a text file

In [ ]:
Plot(gple)
write_data(gple, 'gple_results.dat')